# packages 

In [52]:
import numpy as np

In [71]:
def get_security_transition_matrix():
    # The transition matriw is the same regardless tif the game is cycled or not
    n_squares = 15 
    T_m = np.zeros((n_squares+1, n_squares+1)) 
    #Il y a le i=0 qui n'est pas utilisé, comme ça, ça permet d'avoir l'index = numéro de square
    for i in range(1,n_squares):
        T_m[i][i] = 1/2
        if(i==3):
            T_m[i][i+1] = 1/4 #if takes slow line
            T_m[i][11] = 1/4 #if takes fast line
        elif(i==10):
            T_m[i][15] = 1/2            
        else:
            T_m[i][i+1] = 1/2
        T_m[15][15] = 1
    return T_m

def get_normal_transition_matrix(layout, cycle:bool):
    # The transition matriw is the same regardless tif the game is cycled or not
    #If the player just passes through square 3 without stopping, 
    # he continues to square 4 or beyond (5, 6, etc), as if the other path does not exist
    n_squares = 15 
    T_m = np.zeros((n_squares+1, n_squares+1))
    for i in range(1,n_squares):
        T_m[i][i] = 1/3
        if(i==3):
            # #if takes slow line
            T_m[i][i+1] = 1/6 
            T_m[i][i+2] = 1/6
            #if takes fast line
            T_m[i][11] = 1/6
            T_m[i][12] = 1/6

        elif(i==10):
            if(cycle): 
                T_m[i][15] = 1/3
                T_m[i][1] = 1/3
            else:
                T_m[i][15] = 2/3

        elif(i==14):
            if(cycle):
                T_m[i][1] = 1/3
                T_m[i][1] = 1/3
            else:
                T_m[i][15] = 2/3
        else:
            T_m[i][i+1] = 1/3
            T_m[i][i+2] = 1/3
    T_m[n_squares][n_squares] = 1

    
    # Il faut encore prendre les pièges et bonuses en compte 
    #il faut reparcourir Chaque élément de la T_m
    for i in range(1,n_squares):
        for j in range(1,n_squares):
                # une chance sur 2 de trigger le piège ou bonus
                if(layout[j]==1): # type 1: go back at 1st square
                    T_m[i][1] += T_m[i][j]/2 # il faut transférer une demi proba vers le square 1
                    T_m[i][j] /=2
                elif (layout==2): #type 2: go back 3 squares
                    #il y a qlq cas spéciaux
                    if (j>=11 and j <=13): # if j ==11 12 or 13 --> go to 1,2 or 3 with proba 1/2
                        #diff = 10
                        T_m[i][j-10] += T_m[i][j]/2
                        T_m[i][j] /=2
                    # si trap type 2 à la case 2 ou 3, on peut pas aller moins loin que le start 
                    elif(j==1 or j == 2):
                        T_m[i][1] += T_m[i][j]/2 # il faut transférer une demi proba vers le square 1
                        T_m[i][j] /=2
                    else: # cas "commun"
                        T_m[i][j-3] += T_m[i][j]/2
                        T_m[i][j] /=2
    return T_m

def get_risky_transition_matrix(layout, cycle:bool):
    pass

In [72]:
class markovDecision():
    def __init__(self, layout:  np.ndarray, cycle:bool):
        self.security_transition_matrix = get_security_transition_matrix()
        self.normal_transition_matrix = get_normal_transition_matrix(layout,cycle)
        self.get_risky_transition_matrix = get_risky_transition_matrix(layout,cycle)
        

In [ ]:
class Board():
    # here we place class attribute (static members)
    # init 
    def __init__(self, layout:  np.ndarray, cycle=False):
        self.layout = layout
        self.cycle = cycle
        # self.squares = np.ones(15) # matrice 15 x 3 avec les actions et expected costs associés. 
        self.position = 0 #position actuelle du joueur
    
    # regular step
    def step(self, size):
        self.position += size

    # reset (in case of circular)
    def reset_position(self):
        self.position = 0

    def get_position(self):
        return self.position

In [54]:
class Dice():
    def __init__(self, type:str):
        self.last_toss = None

        if(type == 'security'):
            self.steps = np.array([0,1])
        elif (type == 'normal'):
            self.steps = np.array([0,1,2])
        elif (type== 'risky'):
            self.steps = np.array([0,1,2,3])
        else: raise ValueError(' /!\  Dice type not existing in current settings')
        

    def roll(self):
        """ 
        draw uniformly a step from the dice
        """
        self.last_toss =  int(np.random.choice(self.steps, size=1))
        return self.last_toss
    
    def get_current_toss(self):
        if(self.last_toss==None): raise ValueError(' /!\ Dice should have been tossed at least once')
        return self.last_toss

In [ ]:
def get_proba_next_move_position(v_k, T_m):
    # v_k = current_state proba vector
    # return next state proba vector
    return np.dot(v_k, T_m)

In [66]:
v_0 = np.zeros(16)
v_0[1] = 1
security_transition_matrix = get_security_transition_matrix()
np.dot(v_0, security_transition_matrix) # représente la proba de la prochaine position

array([0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. ])